In [1]:
#########################################################################################################################
# Kurs: Data science, studia zaoczne, Informatyka w Biznesie
# Data ostatniej poprawki: 04.12.2018
# Autorzy: Mariusz Czarny, Juliusz Sadowski, Anita Koszalko

# Celem projektu jest zastosowanie języka Python przy użyciu bibliotek pandas, nympy, itp., aby wykonać analizę danych.
# Danymi źródłowymi są pliki tekstowe: campaigns.txt, customers.txt, orderlines.txt, orders.txt, products.txt pobrane z
# serwera: https://drive.google.com/drive/folders/0B1A_5ELLOtW4VXgwOG1oZmhpS0k
# Ścieżka źródłowa dla plików: powinny znajdować się w tym samym katalogu co plik projektowy
# Wersja pythona: 3.6.3
# Plik dostępny jest także pod linkiem: 
# macierz korelacji
# funckja celu
# predykcja sprzedazy
# probka testowa/uczaca
# regresja liniowa/logistyczna

# User story: 
#########################################################################################################################

In [2]:
# import wewenętrznej bazy danych
import sqlite3

In [3]:
# import standardowych bibliotek
import pandas as pd
import numpy as np
import os

In [4]:
# import bibliotek służących do wizualizacji
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [5]:
# sprawdza aktualną ścieżkę
%pwd

'C:\\Users\\Mariusz\\studia\\3 semestr\\Data science\\my project'

In [6]:
# połączenie z bazą i wykorzystanie kursora
dbConnection = sqlite3.connect("bookdb");
dbCursor = dbConnection.cursor()

In [7]:
# rozszerzenie plików zawierających dane do wczytania
txtExtention = ".txt"

In [8]:
# tablica z nazwami plików do oczytu, zawierające dane, które będą importowane do tabel bazodanowych
tables = ["Customers", "Orderlines", "Products", "Orders", "Campaigns"]

In [9]:
# wczytywanie kolejnych plików w pętli for
for table in tables:
    # ustawienie scieżki (nazwy plików pisane małymy literami)
    path = os.path.expanduser(table.lower() + txtExtention)
    # wczytywanie pliku używając delimitera tabulacji oraz kodowania 'latin1'
    file = pd.read_csv(path, delimiter="\t", encoding = 'latin1')
    # tworzy na podstawie pliku tabele i wypełnia ją rekordami, jesli istnieje zostaje nadpisana
    file.to_sql(table, dbConnection, if_exists="replace");

In [10]:
ordersSelect = '''SELECT totalprice, campaignid, customerid, orderid from orders '''
orders = pd.read_sql_query(ordersSelect, dbConnection)
orders.head()

,totalprice,campaignid,customerid,orderid
0,190.00,2141,45978,1002854
1,10.00,2173,125381,1002855
2,35.22,2141,103122,1002856
3,10.00,2173,130980,1002857
4,10.00,2141,48553,1002886


In [11]:
customersSelect = '''SELECT customerid, gender from customers '''
customers = pd.read_sql_query(customersSelect, dbConnection)

In [12]:
customers.head()

,customerid,gender
0,174596,M
1,68239,M
2,10203,F
3,174288,F
4,68099,M


In [13]:
ordersCustomer = orders.merge(customers, on='customerid')

In [14]:
ordersCustomer.head()

,totalprice,campaignid,customerid,orderid,gender
0,190.00,2141,45978,1002854,M
1,10.00,2173,125381,1002855,M
2,35.22,2141,103122,1002856,None
3,10.00,2173,130980,1002857,F
4,10.00,2141,48553,1002886,None


In [15]:
selectCampaigns = '''SELECT discount, campaignid from campaigns '''
campaignsDF = pd.read_sql_query(selectCampaigns, dbConnection)
campaignsDF.sort_values(by=["discount"], ascending=0).head(100)

,discount,campaignid
97,50,2098
96,50,2097
65,50,2066
72,50,2073
22,50,2023
187,50,2188
21,45,2022
184,40,2185
37,32,2038
94,30,2095


In [16]:
ordersCustomerAndCampaigns = ordersCustomer.merge(campaignsDF, on='campaignid')
ordersCustomerAndCampaigns.head()

,totalprice,campaignid,customerid,orderid,gender,discount
0,190.00,2141,45978,1002854,M,10
1,35.22,2141,103122,1002856,None,10
2,10.00,2141,48553,1002886,None,10
3,10.00,2141,130970,1003044,F,10
4,17.50,2141,21927,1003046,F,10


In [17]:
orderLinesSelect = '''SELECT orderlineid, orderid, productid, billdate from Orderlines '''
orderLines = pd.read_sql_query(orderLinesSelect, dbConnection)

In [18]:
orderLines.head()

,orderlineid,orderid,productid,billdate
0,1010561,1006414,10834,2011-03-08 00:00:00
1,1010562,1006541,11052,2011-01-20 00:00:00
2,1010563,1006542,11070,2011-01-20 00:00:00
3,1010564,1010154,11196,2009-11-20 00:00:00
4,1010565,1009110,11048,2009-11-12 00:00:00


In [19]:
orderlinesOrderCustomerCampaign = orderLines.merge(ordersCustomerAndCampaigns, on='orderid')

In [20]:
orderlinesOrderCustomerCampaign.head(20)

,orderlineid,orderid,productid,billdate,totalprice,campaignid,customerid,gender,discount
0,1010561,1006414,10834,2011-03-08 00:00:00,36.00,2141,48167,M,10
1,1010560,1006414,10842,2011-01-20 00:00:00,36.00,2141,48167,M,10
2,1010562,1006541,11052,2011-01-20 00:00:00,20.00,2141,15824,M,10
3,1010563,1006542,11070,2011-01-20 00:00:00,19.12,2141,124842,M,10
4,1010564,1010154,11196,2009-11-20 00:00:00,14.95,2173,49720,M,0
5,1010565,1009110,11048,2009-11-12 00:00:00,49.90,2141,102019,F,10
6,1010566,1009110,11196,2009-11-20 00:00:00,49.90,2141,102019,F,10
7,1010567,1008491,10816,2009-11-04 00:00:00,175.00,2141,119644,F,10
8,1010568,1008492,11048,2009-11-11 00:00:00,10.00,2173,105604,M,0
9,1010569,1010189,11196,2009-11-20 00:00:00,14.95,2173,108223,F,0


In [21]:
orderlinesOrderCustomerCampaign["billdate"] = pd.to_datetime(orderlinesOrderCustomerCampaign["billdate"]).dt.year

In [22]:
orderlinesOrderCustomerCampaign = orderlinesOrderCustomerCampaign.drop(columns=["orderlineid", "orderid", "productid", "customerid", "campaignid"])

In [23]:
orderlinesOrderCustomerCampaign.head(20)

,billdate,totalprice,gender,discount
0,2011,36.00,M,10
1,2011,36.00,M,10
2,2011,20.00,M,10
3,2011,19.12,M,10
4,2009,14.95,M,0
5,2009,49.90,F,10
6,2009,49.90,F,10
7,2009,175.00,F,10
8,2009,10.00,M,0
9,2009,14.95,F,0


In [24]:
orderlinesOrderCustomerCampaign["gender"] = pd.get_dummies(orderlinesOrderCustomerCampaign["gender"])

In [25]:
orderlinesOrderCustomerCampaign.head(20)

,billdate,totalprice,gender,discount
0,2011,36.00,0,10
1,2011,36.00,0,10
2,2011,20.00,0,10
3,2011,19.12,0,10
4,2009,14.95,0,0
5,2009,49.90,1,10
6,2009,49.90,1,10
7,2009,175.00,1,10
8,2009,10.00,0,0
9,2009,14.95,1,0


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB

model = LinearRegression(fit_intercept=True)
model = GaussianNB()

In [38]:
model.fit(x[, np.newaxis], y)


xfit = np.linspace(0, 10, 1000)
yfit = model.predict(xfit)

plt.scatter(x, y)
plt.plot(xfit, yfit);

SyntaxError: invalid syntax (<ipython-input-38-da6f4e4e8132>, line 3)